In [ ]:
!pip install FlexCode[all]

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.8 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
!pip install zuko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00


In [ ]:
from enum import Enum, auto

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

import sklearn

from argparse import ArgumentParser
import logging
import torch.optim as optim

import numpy as np
import scipy.stats
import flexcode
from flexcode.regression_models import NN
from flexcode.regression_models import RandomForest
import matplotlib.pyplot as plt

import zuko

In [ ]:
class NoiseType(Enum):
    DIAGONAL = auto()
    ISOTROPIC = auto()
    ISOTROPIC_ACROSS_CLUSTERS = auto()
    FIXED = auto()


class MixtureDensityNetwork(nn.Module):
    """
    Mixture density network.

    [ Bishop, 1994 ]

    Parameters
    ----------
    dim_in: int; dimensionality of the covariates
    dim_out: int; dimensionality of the response variable
    n_components: int; number of components in the mixture model
    """

    def __init__(self, dim_in, dim_out, n_components, hidden_dim, noise_type=NoiseType.DIAGONAL, fixed_noise_level=None):
        super().__init__()
        assert (fixed_noise_level is not None) == (
            noise_type is NoiseType.FIXED)
        num_sigma_channels = {
            NoiseType.DIAGONAL: dim_out * n_components,
            NoiseType.ISOTROPIC: n_components,
            NoiseType.ISOTROPIC_ACROSS_CLUSTERS: 1,
            NoiseType.FIXED: 0,
        }[noise_type]
        self.dim_in, self.dim_out, self.n_components = dim_in, dim_out, n_components
        self.noise_type, self.fixed_noise_level = noise_type, fixed_noise_level
        self.pi_network = nn.Sequential(
            nn.Linear(dim_in, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_components),
        )
        self.normal_network = nn.Sequential(
            nn.Linear(dim_in, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, dim_out * n_components + num_sigma_channels)
        )

    def forward(self, x, eps=1e-6):
        #
        # Returns
        # -------
        # log_pi: (bsz, n_components)
        # mu: (bsz, n_components, dim_out)
        # sigma: (bsz, n_components, dim_out)
        #
        log_pi = torch.log_softmax(self.pi_network(x), dim=-1)
        normal_params = self.normal_network(x)
        mu = normal_params[..., :self.dim_out * self.n_components]
        sigma = normal_params[..., self.dim_out * self.n_components:]
        if self.noise_type is NoiseType.DIAGONAL:
            sigma = torch.exp(sigma + eps)
        if self.noise_type is NoiseType.ISOTROPIC:
            sigma = torch.exp(sigma + eps).repeat(1, self.dim_out)
        if self.noise_type is NoiseType.ISOTROPIC_ACROSS_CLUSTERS:
            sigma = torch.exp(sigma + eps).repeat(1,
                                                  self.n_components * self.dim_out)
        if self.noise_type is NoiseType.FIXED:
            sigma = torch.full_like(mu, fill_value=self.fixed_noise_level)
        mu = mu.reshape(-1, self.n_components, self.dim_out)
        sigma = sigma.reshape(-1, self.n_components, self.dim_out)
        return log_pi, mu, sigma

    def loss(self, x, y):
        log_pi, mu, sigma = self.forward(x)
        z_score = (y.unsqueeze(1) - mu) / sigma
        normal_loglik = (
            -0.5 * torch.einsum("bij,bij->bi", z_score, z_score)
            - torch.sum(torch.log(sigma), dim=-1)
        )
        loglik = torch.logsumexp(log_pi + normal_loglik, dim=-1)
        return -loglik

    def sample(self, x):
        log_pi, mu, sigma = self.forward(x)
        cum_pi = torch.cumsum(torch.exp(log_pi), dim=-1)
        rvs = torch.rand(len(x), 1).to(x)
        rand_pi = torch.searchsorted(cum_pi, rvs)
        rand_normal = torch.randn_like(mu) * sigma + mu
        samples = torch.take_along_dim(
            rand_normal, indices=rand_pi.unsqueeze(-1), dim=1).squeeze(dim=1)
        return samples

In [ ]:
def generate_data(n_draws,d):
    x = np.random.normal(0, 1, n_draws*d)
    x = x.reshape((n_draws,d))
    y = np.random.normal(x[:,0], 1, n_draws)
    return x, y.reshape((len(y), 1))

d=50
x_train, y_train = generate_data(10000,d)
x_validation, y_validation = generate_data(10000,d)
x_test, y_test = generate_data(10000,d)

In [ ]:
n_inter = 200
x = torch.Tensor(x_train)
y = torch.Tensor(y_train)

mx_model = MixtureDensityNetwork(d, 1, n_components=3, hidden_dim=50, noise_type=NoiseType.DIAGONAL)
optimizer = optim.Adam(mx_model.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_inter)

for i in range(n_inter):
    optimizer.zero_grad()
    loss = mx_model.loss(x, y).mean()
    loss.backward()
    optimizer.step()
    scheduler.step()

In [ ]:
# Parameterize model
fx_model = flexcode.FlexCodeModel(RandomForest, max_basis=5, basis_system="cosine")

# Fit and tune model
fx_model.fit(x_train, y_train)
fx_model.tune(x_validation, y_validation)

In [ ]:
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow = zuko.flows.NSF(1, d, transforms=3, hidden_features=[128] * 3)

# Train to maximize the log-likelihood
optimizer = torch.optim.AdamW(flow.parameters(), lr=1e-3)

for i in range(n_inter):
    loss = -flow(torch.Tensor(x_train)).log_prob(torch.Tensor(y_train))  # -log p(y | x)
    loss = loss.mean()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### Gaussian Mixture

In [ ]:
x_test = torch.Tensor(x_test)
log_pi, mu, sigma = mx_model.forward(x_test)

log_pi=log_pi.detach().numpy()
mu_mat=mu.detach().numpy().reshape(mu.size()[0],mu.size()[1])
sigma_mat=sigma.detach().numpy().reshape(mu.size()[0],mu.size()[1])
y_grid = np.linspace(-6, 6, 200)

results = np.empty((mu.size()[0], mu.size()[1], y_grid.size))

# Iterate over each new sample
for i in range(mu.size()[0]):
    aux_mu = mu_mat[i, :]
    aux_sigma = sigma_mat[i, :]
    aux_log_pi = log_pi[i, :]
    z_score = (y_grid - aux_mu[:, np.newaxis])/aux_sigma[:, np.newaxis]
    aux_log_pi=aux_log_pi[:, np.newaxis]
    loglik = aux_log_pi - (1/2)*z_score**2 - np.log(np.sqrt(np.pi*2)*aux_sigma[:, np.newaxis])

    # Store the result in the results array
    results[i, :, :] = np.exp(loglik)

densities=np.sum(results, axis=1)

### FlexCode

In [ ]:
fx_densities, fx_y_grid = fx_model.predict(x_test, n_grid=200)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


### Normalizing Flows


In [ ]:
densities_flow = np.empty((x_test.size()[0], y_grid.size))
for i in range(x_test.size()[0]):
    x_test_rep = np.repeat(x_test[i,:].detach().numpy()[:, np.newaxis].T, repeats=y_grid.size, axis=0)
    flow_test=flow(torch.Tensor(x_test_rep))
    densities_flow[i, :] = np.exp(flow_test.log_prob(torch.Tensor(y_grid[:, np.newaxis])).detach().numpy())

## Plots

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats
from matplotlib.backends.backend_pdf import PdfPages
import os

# Custom theme settings to match ggplot2's theme_bw but with a closed box
plt.rcParams.update({
    'axes.edgecolor': 'black',
    'axes.linewidth': 0.8,
    'axes.grid': True,
    'grid.color': '#D3D3D3',
    'grid.linestyle': '-',  # Solid lines for grid
    'grid.linewidth': 0.6,
    'grid.alpha': 0.7,
    'axes.facecolor': 'white',  # White background
    'axes.spines.top': True,  # Enable top spine
    'axes.spines.right': True,  # Enable right spine
    'font.size': 16,  # Reduced font size for cleaner look
    'xtick.bottom': True,
    'ytick.left': True,
})

# Define the custom colors
color1 = "#1E88E5"  # Blue
color2 = "#D81B60"  # Pink
color3 = "#44CA2E"  # Green

# Number of subplots
num_plots = 6
rows = (num_plots + 1) // 2  # Calculate number of rows needed for 2 panels per row

# Create subplots with a shared legend
fig, axs = plt.subplots(rows, 2, figsize=(16, 4 * rows), sharex=True)

# Adjust the layout to make space for the legend at the top
plt.subplots_adjust(top=0.85, bottom=0.1, left=0.1, right=0.9, hspace=0.4)

axs = axs.flatten()  # Flatten the array of axes for easy iteration

# Plot each panel with increased line width
for ii in range(num_plots):
    true_density = scipy.stats.norm.pdf(y_grid, x_test[ii, 0], 1)

    axs[ii].plot(y_grid, densities[ii, :], color=color1, label="Gaussian Mixture", linewidth=3)
    axs[ii].plot(fx_y_grid, fx_densities[ii, :], color=color2, label="FlexCode", linewidth=3)
    axs[ii].plot(fx_y_grid, densities_flow[ii, :], color=color3, label="NFlow", linewidth=3)
    axs[ii].plot(y_grid, true_density, color="black", label="True Density", linewidth=3)

    # Custom grid settings
    axs[ii].grid(True, which='major', color='#D3D3D3', linestyle='-', linewidth=0.6)
    axs[ii].grid(False, which='minor')

    # Add x and y labels
    axs[ii].set_xlabel('y')
    axs[ii].set_ylabel('f(y|x)')

# Hide any unused subplots
for ax in axs[num_plots:]:
    ax.set_visible(False)

# Create a single legend for all subplots with smaller font size, placed closer to the plot area at the top
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.97), fontsize=16, ncol=3, frameon=False)

# Save the figure to a PDF file
with PdfPages('/content/plot_densities_gaussian.pdf') as pdf:
    pdf.savefig(fig, bbox_inches='tight')
    plt.close(fig)

# Move the file to Google Drive
os.system('mv /content/plot_densities_gaussian.pdf /content/gdrive/')


256

## Compute L2 loss

In [ ]:
import sys
sys.path.append('/content/cdetools/python/src')
!pip install git+https://github.com/lee-group-cmu/cdetools.git#subdirectory=python


  Cloning https://github.com/lee-group-cmu/cdetools.git to /tmp/pip-req-build-e7i1buzi
  Running command git clone --filter=blob:none --quiet https://github.com/lee-group-cmu/cdetools.git /tmp/pip-req-build-e7i1buzi
  Resolved https://github.com/lee-group-cmu/cdetools.git to commit 4bd829c0888bf3d3cf346eb60199bd784880c979
  Preparing metadata (setup.py) ... done
  Created wheel for cdetools: filename=cdetools-0.0.2-py3-none-any.whl size=6653 sha256=4156a222e301fc528a5b460794380cf0160b06b15ca1361562758a9689eecf0c
  Stored in directory: /tmp/pip-ephem-wheel-cache-u9piq1x3/wheels/72/f7/6a/e7648014b50f19fc040b9f7a4ac5eb5c15f4c0eb31fe75fcf2
Successfully built cdetools


In [ ]:
import pandas as pd
import cdetools as cdetools

# Assuming the densities, y_grid, fx_densities, fx_y_grid, densities_flow, and y_test are defined

# Calculate loss and standard error for each method
gm_loss = cdetools.cde_loss(densities, y_grid, y_test)  # Gaussian Mixture
fx_loss = cdetools.cde_loss(fx_densities, fx_y_grid, y_test)  # FlexCode
nf_loss = cdetools.cde_loss(densities_flow, fx_y_grid, y_test)  # NFlow

# Create a DataFrame with loss and standard error formatted as "loss (standard error)"
results = pd.DataFrame({
    'Gaussian Mixture': [f"{gm_loss[0]:.4f} ({gm_loss[1]:.4f})"],
    'FlexCode': [f"{fx_loss[0]:.4f} ({fx_loss[1]:.4f})"],
    'NFlow': [f"{nf_loss[0]:.4f} ({nf_loss[1]:.4f})"]
})

# Transpose the DataFrame
results_transposed = results.T
results_transposed.columns = ['Loss (Standard Error)']

# Convert the transposed DataFrame to LaTeX format
latex_code = results_transposed.to_latex(header=True, index=True)

# Display the LaTeX code
print(latex_code)


\begin{tabular}{ll}
\toprule
 & Loss (Standard Error) \\
\midrule
Gaussian Mixture & 0.1154 (0.0121) \\
FlexCode & -0.2657 (0.0015) \\
NFlow & 0.1430 (0.0083) \\
\bottomrule
\end{tabular}



In [ ]:
pit_gaussian = cdetools.cdf_coverage(densities, y_grid, y_test)  # Gaussian Mixture
pit_flexcode = cdetools.cdf_coverage(fx_densities, fx_y_grid, y_test)  # FlexCode
pit_nflow = cdetools.cdf_coverage(densities_flow, fx_y_grid, y_test)  # NFlow

array([0.92804484, 0.55958513, 0.59170408, ..., 0.76992398, 0.58735878,
       0.2323181 ])

## PIT statistic

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
from matplotlib.backends.backend_pdf import PdfPages

def pit_histogram(ax, values, ci_level, n_bins=30, title=None):
    n = values.shape[0]
    ci_quantity = (1 - ci_level) / 2
    low_lim = binom.ppf(q=ci_quantity, n=n, p=1/n_bins)
    upp_lim = binom.ppf(q=ci_level + ci_quantity, n=n, p=1/n_bins)

    # Plot histogram with grey bars
    ax.hist(values, bins=n_bins, color='grey')

    # Add confidence interval lines and shaded area with color #1E88E5
    ax.axhline(y=low_lim, color='grey')
    ax.axhline(y=upp_lim, color='grey')
    ax.axhline(y=n/n_bins, label='Uniform Average', color='red')
    ax.fill_between(x=np.linspace(0, 1, 100),
                    y1=np.repeat(low_lim, 100),
                    y2=np.repeat(upp_lim, 100),
                    color='#1E88E5', alpha=0.2)

    # Set the x-axis limit to be between 0 and 1
    ax.set_xlim(0, 1)

    if title is not None:
        ax.set_title(title, size=22)

    ax.set_xlabel("PIT Value", size=16)
    ax.set_ylabel("Frequency", size=16)

# Generate the PDF with all panels and a single legend
with PdfPages("/content/pit_histograms_panels_single_legend.pdf") as pdf:
    # Create a figure to hold the panels
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # 1 row, 3 columns

    # Add each PIT plot to a panel
    for ax, pit_data, title in zip(axes, [pit_gaussian, pit_flexcode, pit_nflow], ["Gaussian Mixture", "FlexCode", "NFlow"]):
        pit_histogram(ax, pit_data, 0.95, n_bins=100, title=title)

    # Create a single legend for all subplots
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=3, prop={'size': 16})

    # Adjust layout and save to the PDF
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close()

print("PDF saved successfully!")


PDF saved successfully!


In [ ]:
from scipy.stats import kstest


# Perform KS test for uniformity on each PIT dataset
ks_gaussian = kstest(pit_gaussian, 'uniform', args=(0, 1))
ks_flexcode = kstest(pit_flexcode, 'uniform', args=(0, 1))
ks_nflow = kstest(pit_nflow, 'uniform', args=(0, 1))

ks_results = {
    'Gaussian Mixture': ks_gaussian,
    'FlexCode': ks_flexcode,
    'NFlow': ks_nflow
}
ks_df = pd.DataFrame({
    "Model": ["Gaussian Mixture", "FlexCode", "NFlow"],
    "KS Statistic": [ks_gaussian.statistic, ks_flexcode.statistic, ks_nflow.statistic],
    "p-value": [ks_gaussian.pvalue, ks_flexcode.pvalue, ks_nflow.pvalue]
})
ks_df

,Model,KS Statistic,p-value
0,Gaussian Mixture,0.145554,2.326410e-185
1,FlexCode,0.055588,2.673130e-27
2,NFlow,0.268852,0.000000e+00
